### Set up

In [1]:
import fxcmpy
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
%matplotlib inline
from pyti.exponential_moving_average import exponential_moving_average as ema

In [2]:
pip_cost = .0879
lot_size = 10
ema_fast = 51
ema_slow = 200

### Connect

In [3]:
con = fxcmpy.fxcmpy(config_file='fxcm.config')

### Get some data into a DataFrame

In [4]:
df = con.get_candles('GBP/JPY', 
                     period = 'D1', 
                     start = dt.datetime(2017, 1, 1), 
                     end = dt.datetime(2018, 12, 1))

### Calculate the EMA's and add them to the dataframe

In [5]:
df['ema_fast'] = ema(df['askclose'], ema_fast)
df['ema_slow'] = ema(df['askclose'], ema_slow)

### Detect the EMA crossovers and add a col for this in the dataframe 

In [6]:
df['position'] = np.where(df['ema_fast'] > df['ema_slow'],1,0)

### Add a signal

In [7]:
df['signal'] = df['position'].diff()

### To begin building our backtester...
A quick and easy way to find rolling returns is to simply get the overall profit or loss for each trading day and storing it in our DataFrame “df”. To do so, we’ll simply get the difference between the open and close of the day and multiply this value by 100 to get the daily return into pips.

**Note:**   We are using the value of 100 to convert the daily returns into pips because our system is currently trading a JPY based currency pair.  This value may need to be modified if you are trading different instruments.

In [8]:
df['difference (pips)'] = (df['askclose'] - df['askopen']) * 100

### Create a new column in our DataFrame called “total”. 
This column will store a running profit/loss amount for our strategy.  In order to populate this field, we will iterate through our DataFrame and, when we have a trading signal, we will add the daily profit/loss (converted to USD by using our pip_cost and lot_size variable) to the “total” column for each day that the signal is active. For all other days we will keep the total the same.


In [9]:
returns = 0
CountPL=False
for i, row in df.iterrows():
    if CountPL==True:
        returns += (row['difference (pips)'] * pip_cost * lot_size)
        df.loc[i,'total'] = returns
    else:
        df.loc[i,'total'] = returns

    if row['position'] == 1:
        CountPL=True
    else:
        CountPL=False

### Visualise

In [10]:
# Create an empty grapth
#fig = plt.figure(figsize=(14,8))

# Add an axis to the empty graph
#ax1 = fig.add_subplot(111,  ylabel='GBP/JPY Price')

# Plot the closing ask price on that axis
#df['askclose'].plot(ax=ax1, color='r', lw=1)

# Plot the moving averages
#df[['ema_fast']].plot(ax=ax1, lw=2)
#df[['ema_slow']].plot(ax=ax1, lw=2)

df.loc[df.position == 1.0]

#ax1.plot( df.loc[df.position == 1.0].index, df.ema_fast[df.position == 1.0], '^', markersize=10, color='m' )

#ax1.plot(df.loc[df.position == -1.0].index, df.ema_slow[df.position == -1.0],'v', markersize=10, color='k')

#ax2 = ax1.twinx()
#ax2.set_ylabel('Profits in $')
#ax2.plot(df['total'], color = 'green')

#plt.show()

,bidopen,bidclose,bidhigh,bidlow,askopen,askclose,askhigh,asklow,tickqty,ema_fast,ema_slow,position,signal,difference (pips),total
date,,,,,,,,,,,,,,,
2017-08-24 21:00:00,139.527,140.207,140.432,139.298,139.572,140.262,140.453,139.321,262803,143.418102,142.879015,1,1.0,69.0,0.0000
2017-08-25 21:00:00,140.207,140.693,140.841,140.126,140.262,140.918,140.918,140.163,301856,143.303517,142.854450,1,0.0,65.6,57.6624
2017-08-27 21:00:00,140.693,141.036,141.373,140.811,140.918,141.220,141.554,140.935,1128,143.199261,142.832926,1,0.0,30.2,84.2082
2017-08-28 21:00:00,141.036,141.242,141.412,140.429,141.220,141.324,141.435,140.456,197641,143.099566,142.814694,1,0.0,10.4,93.3498
2017-08-29 21:00:00,141.242,141.736,141.993,140.006,141.324,141.836,142.014,140.031,437120,143.022670,142.801809,1,0.0,51.2,138.3546
2017-08-30 21:00:00,141.736,142.451,142.770,141.554,141.836,142.511,142.791,141.599,322284,142.982884,142.797569,1,0.0,67.5,197.6871
2017-08-31 21:00:00,142.451,142.158,142.897,141.540,142.511,142.234,142.919,141.563,347475,142.926837,142.793580,1,0.0,-27.7,173.3388
2017-09-01 21:00:00,142.158,142.742,143.001,142.003,142.234,142.894,143.066,142.120,323541,142.903806,142.797490,1,0.0,66.0,231.3528
2017-09-03 21:00:00,142.742,141.903,141.903,141.256,142.894,142.026,142.061,141.408,1880,142.843132,142.791501,1,0.0,-86.8,155.0556
